In [1]:
import tensorflow as tf
from tensorflow.keras.metrics import binary_crossentropy
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet import ResNet50


def unstack(tensor, axis=3, image_size=512):
    unstacked_tensors = []
    for i, t in enumerate(tf.unstack(tensor, axis=axis)):
        unstacked_tensors.append(tf.reshape(t, [-1, image_size, image_size, 1]))
    return unstacked_tensors



def gt_loss(y_true, y_pred, max_degree=6, image_size=512):
    y_true = unstack(y_true)
    y_pred = unstack(y_pred)
    
    batch_size = tf.shape(y_true[0])[0]

    soft_mask = tf.reshape(y_true[0], (batch_size, image_size, image_size))
    #  Crossentropy loss for vertex channel
    pv_loss = tf.reduce_mean(binary_crossentropy(y_true[0], y_pred[0]))
    pe_loss = 0
    for i in range(max_degree):
        pe_crossentropy = binary_crossentropy(y_true[1 + 3*i], y_pred[1 + 3*i])
        # Apply only to keypoints!
        pe_loss += tf.reduce_mean(tf.multiply(soft_mask, pe_crossentropy))
    direction_loss = 0
    for i in range(max_degree):
        v1 = tf.concat(y_true[2 + 3*i:4 + 3*i], axis=3)
        v2 = tf.concat(y_pred[2 + 3*i:4 + 3*i], axis=3)
        # Apply only to keypoints!
        direction_loss += tf.reduce_mean(tf.multiply(y_true[0], tf.square(v2 - v1)))
    return pv_loss + 10 * pe_loss + 1000 * direction_loss


def conv_block(inputs, filters):
    x = layers.Conv2D(filters, 3, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    return x


def decoder_block(inputs, skip_features, filters):
    x = layers.Conv2DTranspose(filters, (2, 2), strides=2, padding='same')(inputs)
    x = layers.Concatenate()([x, skip_features])
    x = conv_block(x, filters)
    return x


class GteOutput(layers.Layer):
    def __init__(self, max_degree=6, *args, **kwargs):
        super(GteOutput, self).__init__(*args, **kwargs)
        self.max_degree = max_degree
        
    def call(self, inputs):
        outputs = unstack(inputs)
        outputs[0] = tf.math.sigmoid(outputs[0])
        for i in range(self.max_degree):
            outputs[1 + 3*i] = tf.math.sigmoid(outputs[1 + 3*i])
        return tf.concat(outputs, axis=3)

    
def resnet50_unet(input_shape, max_degree=6):
    inputs = layers.Input(shape=input_shape, name='input')

    # Pre-trained resnet50
    resnet50 = ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)

    # Skip connections
    s1 = resnet50.get_layer('input').output                   # x1
    s2 = resnet50.get_layer('conv1_relu').output              # x2
    s3 = resnet50.get_layer('conv2_block3_out').output        # x4
    s4 = resnet50.get_layer('conv3_block4_out').output        # x8

    # Bridge
    b1 = resnet50.get_layer('conv4_block6_out').output

    # Decoder blocks
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    
    raw_outputs = layers.Conv2D(1 + 3 * max_degree, kernel_size=(3, 3), padding='same', activation='linear', name='raw_output')(d4)
    
    outputs = GteOutput(max_degree=max_degree, name="outputs")(raw_outputs)
    
    return Model(inputs, outputs, name='resnet50_unet')

In [2]:
import pandas as pd

vegas = pd.read_csv('data/tiles_Vegas.csv', index_col=0)
paris = pd.read_csv('data/tiles_Paris.csv', index_col=0)
shanghai = pd.read_csv('data/tiles_Shanghai.csv', index_col=0)
khartoum = pd.read_csv('data/tiles_Khartoum.csv', index_col=0)

df = pd.concat([vegas, paris, shanghai, khartoum]).reset_index()
df

df_20cities = pd.read_csv('data/20cities_dataset.csv', index_col=0)

In [3]:
import cv2
from gte import GraphTensorEncoder

import pickle
from collections import defaultdict


def to_int(u):
    return int(u[0]), int(u[1])


def load_graph(path):
    graph = defaultdict(set)
    for u, neighbours in pickle.load(open(path, 'rb')).items():
        u = to_int(u)
        for v in neighbours:
            graph[u].add(to_int(v))
    return graph


class Reader:
    def __init__(self, df, image_size):
        self.encoder = GraphTensorEncoder(max_degree=6, image_size=image_size[0], d=25)
        self.df = df
        self.image_size = image_size
        
    def __call__(self):
        for index, row in self.df.iterrows():
            tile = cv2.imread(row['tile'])
            graph = load_graph(row['graph'])
            gt = self.encoder.encode(graph)
            yield tile, gt
            
image_size = (512, 512)
dataset = tf.data.Dataset.from_generator(Reader(df_20cities, image_size),
                                        output_signature=(
                                            tf.TensorSpec(shape=image_size+(3,), dtype=tf.float32),
                                            tf.TensorSpec(shape=image_size+(19,), dtype=tf.float32)
                                        )).cache()\
                                          .batch(8)\
                                          .repeat()\
                                          .prefetch(buffer_size=tf.data.AUTOTUNE)

2022-06-12 09:12:27.250145: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-12 09:12:39.616564: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13579 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:8b:00.0, compute capability: 7.5


In [ ]:
model = resnet50_unet((512, 512, 3), max_degree=6)

scheduler = tf.keras.optimizers.schedules.ExponentialDecay(1e-2, 3000, 0.1)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=gt_loss)

model.fit(dataset, epochs=5, steps_per_epoch=500)

Epoch 1/5


2022-06-12 09:12:59.735047: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200
2022-06-12 09:13:07.961578: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-12 09:13:25.349098: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.93GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-06-12 09:13:25.349174: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.93GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


237/500 [=============>................] - ETA: 7:00 - loss: 3.1164

In [ ]:
encoder = GraphTensorEncoder(max_degree=6, image_size=512, d=25)

In [ ]:
tiles = []
for tile, graph in Reader(df_20cities.sample(20), image_size):
    tiles.append(tile)
tiles = np.array(tiles)